In [34]:
import numpy as np
import torch
import torch.fx
import transformers
from transformers import AutoTokenizer, LlamaForCausalLM
from vllm import LLM, SamplingParams

import openai
import os 
from gymnasium import envs
from tests.utils import get_all_registered_miniwob_envs
import time
import gymnasium as gym
import re
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [35]:
from miniwob.action import (
    create_coord_click_action,
    create_element_click_action,
    create_focus_and_type_action,
    create_type_action,
    create_none_action   
)

In [36]:
print(get_all_registered_miniwob_envs())

['miniwob/bisect-angle-v1', 'miniwob/book-flight-nodelay-v1', 'miniwob/book-flight-v1', 'miniwob/choose-date-easy-v1', 'miniwob/choose-date-medium-v1', 'miniwob/choose-date-nodelay-v1', 'miniwob/choose-date-v1', 'miniwob/choose-list-v1', 'miniwob/circle-center-v1', 'miniwob/click-button-sequence-v1', 'miniwob/click-button-v1', 'miniwob/click-checkboxes-large-v1', 'miniwob/click-checkboxes-soft-v1', 'miniwob/click-checkboxes-transfer-v1', 'miniwob/click-checkboxes-v1', 'miniwob/click-collapsible-2-nodelay-v1', 'miniwob/click-collapsible-2-v1', 'miniwob/click-collapsible-nodelay-v1', 'miniwob/click-collapsible-v1', 'miniwob/click-color-v1', 'miniwob/click-dialog-2-v1', 'miniwob/click-dialog-v1', 'miniwob/click-link-v1', 'miniwob/click-menu-2-v1', 'miniwob/click-menu-v1', 'miniwob/click-option-v1', 'miniwob/click-pie-nodelay-v1', 'miniwob/click-pie-v1', 'miniwob/click-scroll-list-v1', 'miniwob/click-shades-v1', 'miniwob/click-shape-v1', 'miniwob/click-tab-2-easy-v1', 'miniwob/click-tab-2-

# Predefined Function

In [37]:
# Modified click_action function
def click_action1(tc, it, observation): 
    for element in observation["dom_elements"]:
        if (element["tag"] == tc or element["classes"] == tc) and (element["id"] == it or element["text"] == it):
            return create_element_click_action(element["ref"])
    return 'Cannot find in the DOM_element'

In [38]:
# Find the input text box and click on it
def find_input_action(tag, observation):
    for element in obs["dom_elements"]:
        if element["tag"] == tag:
            return create_element_click_action(element["ref"])

In [39]:
# Type Text
def enter_text_action(input_text, observation):
    target = input_text
    return create_type_action(target)

In [40]:
# Click on the screen based on input coordinates
def coord_click_action(x ,y):
    action = create_coord_click_action(x, y)
    return action

In [41]:
# Action function of doing Nothing
def do_nothing_action():
    return create_none_action()

In [42]:
# Count number of text or shape
def count_type(t, observation):
    shape_type = ['rect', 'circle', 'polygon']
    count = 0
    for element in obs['dom_elements']:
        if element['classes'] == 'SVG_CLASS':
            if t == 'text' and element['tag'] == t: 
                count += 1
            elif t == 'shape' and element['tag'] in shape_type:
                count += 1
    return count

# Get DOM_element

In [43]:
useful_tag = {'button', 'text', 'input_time', 'textarea', 'polygon','label','input_password','rect',"tt",\
              'circle' ,'input_password', 'span', 'input_text', 'input_number', 'input_date', 'input_radio', \
              'tspan', 'input_checkbox', 't', 'button', 'h3', 'ul', 'a', 'p', 'div', 'span','input_text'}

useful_classes = {'folder'}

In [44]:
# after switching positions
def get_DOM_element(observation, useful_tag, useful_classes):
    objects_in_the_image = []
    for d in observation['dom_elements']:
        if d['classes'] in useful_classes:
            if d['text'] != '':
                objects_in_the_image.append(d['classes']+' '+d['text'])
            else:
                objects_in_the_image.append(d['classes']+' '+d['id'])
        elif d['tag'] in useful_tag:
            if d['text'] != '':
                objects_in_the_image.append(d['tag']+' '+d['text'])
            else:
                objects_in_the_image.append(d['tag']+' '+d['id'])
                
    return objects_in_the_image

# Regex

In [45]:
class filter_prompt:
    def __init__(self):
        self.click_format = r'.*click_action1\s*\(\s*(?<!\\)[^\\,]+\s*,\s*(?<!\\)[^\\,]+\s*,\s*observation\s*\)$'
        self.click_format1 = r'.*click_action1\s*\(\s*[^\\,]+\s*,\s*[^\\,]+\s*,\s*observation\s*\)|.*click_action1\s*\(\s*[^\\,]+\s*,\s*observation\s*,\s*[^\\,]+\s*\)$'
        self.find_input_format = r'.*find_input_action\s*\(\s*observation\s*\)$'
        self.enter_text_format = r'.*enter_text_action\s*\(\s*(?<!\\)[^\\,]+\s*,\s*observation\s*\)$'
        self.coord_click_format = r'.*coord_click_action\s*\(\s*\d+\s*,\s*\d+\s*\)$'
        self.do_nothing_format = r'.*do_nothing_action\s*\(\s*\)$'
        self.count_format = r'.*count_type\s*\(\s*[^\\]*,\s*observation\s*\)$'
        self.make_sure = r'\s*\b(action|observation)'
        self.obs = r'\s*observation\s*,\s*reward\s*,\s*terminated\s*,\s*truncated\s*,\s*info\s*=\s*env\.step\(([^)]+)\)'
        
    def check_promt(self, expression=None):
        if expression == None:
            raise ValueError("Expression cannot be None")
           
        if not(bool(re.match(self.make_sure, expression, re.IGNORECASE))):
            return 'not_useful'
        
        if bool(re.match(self.obs, expression, re.IGNORECASE)):
            return expression
        
        if '=' in expression:
            exp = expression.split('=')[1].lstrip()
        elif ':' in expression:
            exp = expression.split(':')[1].lstrip()
        else:
            print('Error in generated expression', expression)
            return 'not_useful'
        action_type = exp.split('(')[0].lstrip()
        print(action_type)
        if action_type == 'click_action1':
            x = re.search(self.click_format1, expression)
            if x == None:
                return False
            return x.string
        
        if action_type == 'find_input_action':
            x = re.search(self.find_input_format, expression)
            if x == None:
                return False
            return x.string
        
        if action_type == 'enter_text_action':
            x = re.search(self.enter_text_format, expression)
            if x == None:
                return False
            return x.string
            
        if action_type == 'coord_click_action':
            x = re.search(self.coord_click_format, expression)
            if x == None:
                return False
            return x.string
        
        if action_type == 'do_nothing_action':
            x = re.search(self.do_nothing_format, expression)
            if x == None:
                return False
            return x.string
        
        if action_type == 'count_type':
            x = re.search(self.count_format, expression)
            if x == None:
                return False
            return x.string
        
        return False

In [46]:
p = filter_prompt()

# Llama2

In [47]:
System_Message = """You are designed to generate programs to solve a wide range of complex web interface tasks.
You should be able to generate the program using either one or a composition of predefined action functions 
along with general python codes to solve different tasks. You should not conversate with human in any context.
"""

# Can also set up example to use it later for each task
Function_Description = """
You should only use the functions provided herewith in the function description. 
Here is the list for the pre-defined functions [click_action1, enter_text_action, coord_click_action, count_type]

To use a function, please refer to the Name, Input, Output, Description of the functions, and usage examples below. 
Action functions should be called correctly in the solution. 

Function Name: click_action1
Input: tag or element, id or text, observation
Output: create_click_element_action
Description: useful when you want to click on an element in the web interface. This functions cannot be 
            generalized on names. Normally first input is one of tag or element and second is test or id.
            The output is given as the action by calling click_action1 function or 
            'Cannot find in the DOM_element' if no such thing to be clicked on.
Example: Objects in Image: Button One;
         Task: Click button ONE;
         Solution: action = click_action1('button', 'ONE', observation)
                   observation, reward, terminated, truncated, info = env.step(action)

Function Name: enter_text_action
Input: input_text, observation
Output: create_type_action
Description: useful when you want to type the input_text into input text box or a similar object like input_number
            that can accept 
             text given the observation of the task interface. Need to call click_action1 to click on it before
             calling this function.
             The output is given as the action by calling enter_text_action function.
Example: Objects in Image: input_text textbox; 
         Task: Type 'Heyi' into textbox;
         Solution: ction = click_action1('input_text', 'textbox', observation)
                   observation, reward, terminated, truncated, info = env.step(action)
                   action = enter_text_action('Heyi', observation)
                   observation, reward, terminated, truncated, info = env.step(action)
            

Name: count_type
Input: tag, observation
Output: count
Description: useful when you want count the number with the type tag given by the input. Observation of the 
             task interface is also given. The output is given as the find count of that element by calling 
             count_type function.
Example: Objects in Image: Shape Circle, Shape Square 
         Task: Count number of Shape objects in the screen 
         Solution: num = count(Shape, observation)
"""


Solution_Description ="""
Objects in the image are given by the concatenation of either tag or classes and either text or id. Solution
should be consistent with objects in the image when calling functions.
The final solution should be given after the text 'Solution?' based on the Object in the image and task given, 
and should only give the solution by either calling the predfined functions' names 
with or without using python format data structure. No need to specify extra text solution when giving solution.
Any other texts can only be given as comment in python format.
Assume that you can direclty use observation before calling the function without checking it.

Cannot assume there are other functions beside previous given functions or unknown imformation from the system.
Solution actions will be executed directly, no need to regive solution again as a list, but add 
observation, reward, terminated, truncated, info = env.step(action) after every action. Actions are independent
from each other.

If you cannot directly solve a given task as shown in the examples in function description or as a hierarchy of actions
to achieve the final goal, you should try to first perform a reasonable action which you can do given the objects
in the image and interact with the environment, post that observe the change in objects in the image, 
and then try to solve the task for new objects in the environment. 
Remember, you can add any other python data structure like if-else, for loop, while loop etc to 
generated program to solve the question. You should generate the program based on the objects in the image and use 
your knowledge from your training data on DOM Elements to understand the objects in the image. 
"""


In [48]:
# prompt = "<s>[INST] <<SYS>> {{" + System_Message + " }}<</SYS>> {{" + Function_Description + "}} [/INST] \
# {{" + Solution_Description + "}} [/INST]"

# prompt = f"<s>[INST] <<SYS>> {{ {self.system_message} }}  {{ {self.function_description} }} {{ {self.solution_description} }}<</SYS>> [/INST]"

In [49]:
task_message = """Next task is a multi-step task, directly performing a series of actions may not solve the task.
Need to observe the changes in the objects in image after performing an/any action to see if 
further actions need to be made to solve the task or not. """

In [50]:
class ChatAppLlama:
    def __init__(self, model_pth, device_type, system_message, function_description, solution_description, quantization=True, max_position_embeddings=4096):
        self.model = LlamaForCausalLM.from_pretrained(model_pth, load_in_8bit=quantization, device_map=device_type, max_position_embeddings=max_position_embeddings)
        self.tokenizer = AutoTokenizer.from_pretrained(model_pth)
        self.system_message = system_message
        self.function_description = function_description
        self.solution_description = solution_description
        self.prompt = f"<s>[INST] <<SYS>> {{ {self.system_message} {self.function_description} {self.solution_description} }}<</SYS>>[/INST]"
        self.original_prompt = f"<s>[INST] <<SYS>> {{ {self.system_message} {self.function_description} {self.solution_description} }}<</SYS>>[/INST]"
        
    def get_model(self):
        return self.model, self.tokenizer
    
    def get_prompt(self):
        return self.prompt
    
    def add_prompt(self, message, mode ='user'):
        if mode == 'answer':
            self.prompt += f"{{ {message} }}"
        if mode == "user":
            self.prompt += f"[INST] {{ {message} }} [/INST]"
        if mode == "system":
            self.prompt += f"<s>[INST] <<SYS>> {{ {message} }} <</SYS>>[/INST]"
        if mode == 'start':
            self.prompt += "<s>"
        if mode == 'end':
            self.prompt += "</s>"
        return
    
    def restore_prompt(self):
        self.prompt = self.original_prompt
        return

In [51]:
class ChatAppLlama_2:
    def __init__(self, system_message, function_description, solution_description, quantization=True, max_position_embeddings=4096):
        self.system_message = system_message
        self.function_description = function_description
        self.solution_description = solution_description
        self.prompt_new = f"<s>[INST] <<SYS>> {{ {self.system_message} {self.function_description} {self.solution_description} }}<</SYS>>[/INST]"
        self.original_prompt = f"<s>[INST] <<SYS>> {{ {self.system_message} {self.function_description} {self.solution_description} }}<</SYS>>[/INST]"
        
    def get_model(self):
        return self.model, self.tokenizer
    
    def get_prompt(self):
        return self.prompt_new
    
    def add_prompt_new (self, message, mode ='user'):
        if mode == 'answer':
            self.prompt_new += f"{{ {message} }}"
        if mode == "user":
            self.prompt_new += f"[INST] {{ {message} }} [/INST]"
        if mode == "system":
            self.prompt_new += f"<s>[INST] <<SYS>> {{ {message} }} <</SYS>>[/INST]"
        return
    
    def restore_prompt(self):
        self.prompt_new = self.original_prompt
        return
    

In [52]:
# Only need to initialize it once
llama2 = ChatAppLlama(model_pth="llama/model_2/", device_type='cuda:0', system_message=System_Message, \
                     function_description=Function_Description, solution_description=Solution_Description)

model, tokenizer = llama2.get_model()

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [53]:
llama2_new = ChatAppLlama_2(system_message=System_Message, \
                     function_description=Function_Description, solution_description=Solution_Description)

# Test Code

In [26]:
# llama2 = ChatAppLlama(model_pth="llama/model_2/", device_type='cuda:0', system_message=System_Message, \
#                      function_description=Function_Description, solution_description=Solution_Description)

# model, tokenizer = llama2.get_model()

In [27]:
# env = gym.make("miniwob/click-button-sequence-v1" , render_mode="human")
# observation, info = env.reset()
# objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
# task=observation["utterance"]
# llama2.add_prompt("Objects in Image: {}; \
#                        Task:{}'  \
#                        Solution?".format(objects_in_the_image,task))

In [28]:
# inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
# generate_ids = model.generate(inputs.input_ids, max_length = 1600)
# solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
# if solution.startswith(llama2.get_prompt()[3:]):
#     # Extract the appended part
#     appended_part = solution[len(llama2.get_prompt())-1:]

In [ ]:
# program_list = appended_part.split('\n') 
# p = filter_prompt()

# for code in program_list:
    
#     if code==" ":
#         continue
#     code_check = p.check_promt(code)
#     # print(code_check)
#     if code_check == False or code_check == 'not_useful':
#         # print('invalid program: ' + code)
#         continue
#     cleaned_code = code_check.lstrip()
#     print(cleaned_code)
#     exec(cleaned_code)

# Task 1 - Click button Sequence

In [24]:
count = 0
iteration = 1
for i in range(iteration):
    p = filter_prompt()
    llama2.restore_prompt()
    env = gym.make("miniwob/click-button-sequence-v1" , render_mode="human")
    observation, info = env.reset()
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    task=observation["utterance"]
    llama2.add_prompt("Objects in Image: {}; \
                       Task:{}'  \
                       Solution?".format(objects_in_the_image,task))
    
    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length = 1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print("solution:",solution)
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
        temp_1=llama2.get_prompt()
        temp_2=len(llama2.get_prompt())
    
#     print("appended part:",appended_part)
        
    program_list = appended_part.split('\n') 
    p = filter_prompt()

    for code in program_list:

        if code==" ":
            continue
#         print(code)
        code_check = p.check_promt(code)
        # print(code_check)
        if code_check == False or code_check == 'not_useful':
            # print('invalid program: ' + code)
            continue
        cleaned_code = code_check.lstrip()
        # print(cleaned_code)
        try:
            exec(cleaned_code)
        except:
            print('output problem')
        
    if reward > 0:
        count += 1
        reward = 0
        
print(count/iteration)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


solution: [INST] <<SYS>> { You are designed to generate programs to solve a wide range of complex web interface tasks.
You should be able to generate the program using either one or a composition of predefined action functions 
along with general python codes to solve different tasks. You should not conversate with human in any context.
 
You should only use the functions provided herewith in the function description. 
Here is the list for the pre-defined functions [click_action1, enter_text_action, coord_click_action, count_type]

To use a function, please refer to the Name, Input, Output, Description of the functions, and usage examples below. 
Action functions should be called correctly in the solution. 

Function Name: click_action1
Input: tag or element, id or text, observation
Output: create_click_element_action
Description: useful when you want to click on an element in the web interface. This functions cannot be 
            generalized on names. Normally first input is one of 

In [134]:
llama2.get_prompt()[3:]

"[INST] <<SYS>> { You are designed to generate programs to solve a wide range of complex web interface tasks.\nYou should be able to generate the program using either one or a composition of predefined action functions \nalong with general python codes to solve different tasks. You should not conversate with human in any context.\n \nYou should only use the functions provided herewith in the function description. \nHere is the list for the pre-defined functions [click_action1, enter_text_action, coord_click_action, count_type]\n\nTo use a function, please refer to the Name, Input, Output, Description of the functions, and usage examples below. \nAction functions should be called correctly in the solution. \n\nFunction Name: click_action1\nInput: tag or element, id or text, observation\nOutput: create_click_element_action\nDescription: useful when you want to click on an element in the web interface. This functions cannot be \n            generalized on names. Normally first input is on

In [25]:
#used to extract the part of the generated text that was added by the model-- excluding the prompt 
# appended_part = solution[len(llama2.get_prompt())-1:]

# Task 2 - click collaspible

In [31]:
env = gym.make("miniwob/click-collapsible-v1" , render_mode="human")
observation, info = env.reset()

In [30]:
# count = 0
# iteration = 2
# for i in range(iteration):
#     env = gym.make("miniwob/click-collapsible-v1", render_mode="human")
#     observation, info = env.reset()
#     task = observation["utterance"]
#     success = False

#     while not success:
#         objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#         p = filter_prompt()
#         llama2.restore_prompt()
#         llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task))

#         inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
#         generate_ids = model.generate(inputs.input_ids, max_length=1600)
#         solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

#         if solution.startswith(llama2.get_prompt()[3:]):
#             appended_part = solution[len(llama2.get_prompt())-1:]

#         program_list = appended_part.split('\n') 
#         for code in program_list:
#             if code.strip() == "":
#                 continue
#             code_check = p.check_promt(code)
#             if code_check in [False, 'not_useful']:
#                 continue
#             cleaned_code = code_check.strip()
#             try:
#                 exec(cleaned_code)
#             except:
#                 print('output problem')
            
#             observation, reward, terminated, truncated, info = env.step(action)
#             if reward > 0:
#                 count += 1
#                 reward = 0
#                 success = True
#                 break 

# print(count/iteration)

In [37]:
count = 0
iteration = 3
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-hard-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
    llama2.add_prompt(task_message)
    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
    
    program_list = appended_part.split('\n')
    reward = 0

    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
#             print('Invalid program:', code)
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            print("Executing....")
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#                     llama2.restore_prompt()
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=1600)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    print("appended part:", appended_part)
                    new_program_list = appended_part.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "gravida".
code_check: action = click_action1('div', 'wrap', observation)
Executing....
code_check: action = enter_text_action('gravida', observation)
Executing....
code_check: action = click_action1('a', 'gravida', observation)
Executing....
Iteration 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "Sit".
code_check: action = click_action1('div', 'tabs-1', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
code_check: action = enter_text_action('a', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
code_check: action = click_action1('a', 'Sit', observation)
Executing....
Iteration 2


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "vitae".
code_check: action = click_action1('div', 'wrap', observation)
Executing....
code_check: action = enter_text_action('vitae', observation)
Executing....
code_check: action = enter_text_action('vitae', observation)
Executing....
code_check: action = enter_text_action('vitae', observation)
Executing....
code_check: action = enter_text_action('vitae', observation)
Executing....
code_check: action = enter_text_action('vitae', observation)
Executing....
code_check: action = enter_text_action('vitae', observation)
Executing....
code_check: action = enter_text_action('vitae', observation)
Executing....
Success Rate: 0.0


In [48]:
correct_solution_step_1= """action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action)"""
    
correct_solution_step_2= """action = click_action1('a', '3', observation)
observation, reward, terminated, truncated, info = env.step(action)"""

correct_solution_step_3= """action = click_action1('a', '1', observation)
observation, reward, terminated, truncated, info = env.step(action)"""

correct_solution_step_4= """action = click_action1('a', '3', observation)
observation, reward, terminated, truncated, info = env.step(action)"""

correct_solution_step_5= """action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action)"""

correct_solution_step_6= """action = click_action1('a', '3', observation)
observation, reward, terminated, truncated, info = env.step(action)"""

In [55]:
correct_solutions=[correct_solution_step_1 , correct_solution_step_2, correct_solution_step_3, correct_solution_step_4
, correct_solution_step_5, correct_solution_step_6]

In [56]:
correct_solutions

["action = click_action1('a', '2', observation)\nobservation, reward, terminated, truncated, info = env.step(action)",
 "action = click_action1('a', '3', observation)\nobservation, reward, terminated, truncated, info = env.step(action)",
 "action = click_action1('a', '1', observation)\nobservation, reward, terminated, truncated, info = env.step(action)",
 "action = click_action1('a', '3', observation)\nobservation, reward, terminated, truncated, info = env.step(action)",
 "action = click_action1('a', '2', observation)\nobservation, reward, terminated, truncated, info = env.step(action)",
 "action = click_action1('a', '3', observation)\nobservation, reward, terminated, truncated, info = env.step(action)"]

In [50]:
env = gym.make("miniwob/click-tab-2-hard-v1", render_mode="human")
observation, info = env.reset()
exec(correct_solution_step_1)
time.sleep(3)
exec(correct_solution_step_2)
time.sleep(3)
exec(correct_solution_step_3)
time.sleep(3)
exec(correct_solution_step_4)
time.sleep(3)
exec(correct_solution_step_5)
time.sleep(3)
exec(correct_solution_step_6)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [59]:
env = gym.make("miniwob/click-tab-2-hard-v1", render_mode="human")
observation, info = env.reset()
exec(correct_solutions[0])
time.sleep(3)
exec(correct_solutions[1])
time.sleep(3)
exec(correct_solutions[2])
time.sleep(3)
exec(correct_solutions[3])
time.sleep(3)
exec(correct_solutions[4])
time.sleep(3)
exec(correct_solutions[5])

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [60]:
llama2.restore_prompt()
llama2_new.restore_prompt()

In [61]:
count = 0
iteration = 1
incorrect_program_list = []
solution_index = 1
for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-hard-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2_new.restore_prompt()
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
    llama2_new.add_prompt_new("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
    llama2_new.add_prompt_new(correct_solutions[0], mode="answer")
    if llama2_new.get_prompt()[3:].startswith(llama2.get_prompt()[3:]):
        appended_part = llama2_new.get_prompt()[3:][len(llama2.get_prompt())-1:]
        print("========= Matches outside loop ==========")
    print("appended part:",appended_part)
    program_list = appended_part.split('\n')
    print("program_list:", program_list)
    reward = 0

    for code in program_list:
        new_program_list=[]
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        code_check= code_check.replace('{', '').replace('}', '')
        if not code_check or code_check == 'not_useful':
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("========= entering inner loop ========")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    llama2_new.add_prompt_new("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    if solution_index < (len(correct_solutions)-1):
                        next_action = correct_solutions[solution_index]
                        llama2_new.add_prompt_new(next_action, mode="answer")
                        solution_index += 1
                        if llama2_new.get_prompt()[3:100] == llama2.get_prompt()[3:100]:
                            print("======= Matches inside loop ======")
                            parts = llama2_new.get_prompt().split('[/INST]')
                            desired_text = parts[-1].strip()
                            desired_text = desired_text.strip() 
                            desired_text = desired_text.replace('{', '').replace('}', '')
#                             print(desired_text)
                            new_program_list=desired_text.split('\n')
                            print("new_program_list:", new_program_list)
                            for code in new_program_list:
                                if code.strip() == "":
                                    continue
                                code_check = p.check_promt(code)
                                if not code_check or code_check == 'not_useful':
                                    continue
                                try:
                                    cleaned_code = code_check.lstrip()
                                    print("code_check inside loop:",cleaned_code)
                                    exec(cleaned_code)
                                except:
                                    print("error")
    #                         program_list=[]
    #                         program_list.extend(new_program_list)
    #                         appended_part_new = llama2_new.get_prompt()[3:][len(llama2.get_prompt())-1:]
                        else:
                            print("no match found")
#                         
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Task: Switch between the tabs to find and click on the link "cursus".
========= Matches outside loop ==========
appended part: action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action) }
program_list: ["action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) }']
code_check: action = click_action1('a', '2', observation)
code_check: observation, reward, terminated, truncated, info = env.step(action) 
========= entering inner loop ========
===== Matches inside loop ======
 action = click_action1('a', '3', observation)
observation, reward, terminated, truncated, info = env.step(action) 
new_program_list: [" action = click_action1('a', '3', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ']
code_check inside loop: action = click_action1('a', '3', observation)
code_check inside loop: observation, reward, terminated, truncated, info = env.ste

# Proper tested version

In [69]:
count = 0
iteration = 1
incorrect_program_list = []
solution_index = 1
for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-hard-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
#     print('Task:', task)

    llama2.restore_prompt()
    llama2_new.restore_prompt()
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
    llama2_new.add_prompt_new("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
    llama2_new.add_prompt_new(correct_solutions[0], mode="answer")
    if llama2_new.get_prompt()[3:].startswith(llama2.get_prompt()[3:]):
        appended_part = llama2_new.get_prompt()[3:][len(llama2.get_prompt())-1:]
        print("========= Matches outside loop ==========")
#     print("appended part:",appended_part)
    program_list = appended_part.split('\n')
    print("\n ===== Program List:", program_list)
    reward = 0

    for code in program_list:
        print("========= Entering inner loop ========")
        new_program_list=[]
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        code_check= code_check.replace('{', '').replace('}', '')
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            time.sleep(5)
            exec(cleaned_code)
            print("\n ============CODE CHECK:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("\n========= Entering inner loop ========")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    llama2_new.add_prompt_new("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    if solution_index < (len(correct_solutions)-1):
                        print("\n=======solution index: ", solution_index )
                        next_action = correct_solutions[solution_index]
                        llama2_new.add_prompt_new(next_action, mode="answer")
                        solution_index += 1
                        if llama2_new.get_prompt()[3:100] == llama2.get_prompt()[3:100]:
                            print("\n======= Matches inside loop ==========")
                            parts = llama2_new.get_prompt().split('[/INST]')
                            desired_text = parts[-1].strip()
                            desired_text = desired_text.strip() 
                            desired_text = desired_text.replace('{', '').replace('}', '')
                            new_program_list=desired_text.split('\n')
                            print("\n===== New program_list:", new_program_list)
                            program_list.extend(new_program_list)
                        else:
                            print("no match found")
#                         
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


========= Matches outside loop ==========

 ===== Program List: ["action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) }']
========= Entering inner loop ========

 ============CODE CHECK: action = click_action1('a', '2', observation)
========= Entering inner loop ========

 ============CODE CHECK: observation, reward, terminated, truncated, info = env.step(action) 

========= Entering inner loop ========

=======solution index:  1

======= Matches inside loop ==========

===== New program_list: [" action = click_action1('a', '3', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ']
========= Entering inner loop ========

 ============CODE CHECK: action = click_action1('a', '3', observation)
========= Entering inner loop ========

 ============CODE CHECK: observation, reward, terminated, truncated, info = env.step(action) 

========= Entering inner loop ========

=======solution index:  2



In [110]:
count = 0
iteration = 1
incorrect_program_list = []
for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-hard-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2_new.restore_prompt()
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
    llama2_new.add_prompt_new("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
    llama2_new.add_prompt_new(correct_solution_step_1, mode="answer")
#     print(llama2_new.get_prompt())
    if llama2_new.get_prompt()[3:].startswith(llama2.get_prompt()[3:]):
        appended_part = llama2_new.get_prompt()[3:][len(llama2.get_prompt())-1:]
        print("========= Matches outside loop ==========")
    print("appended part:",appended_part)
    program_list = appended_part.split('\n')
    print("program_list:", program_list)
    reward = 0

    for code in program_list:
        new_program_list=[]
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        code_check= code_check.replace('{', '').replace('}', '')
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            exec(cleaned_code)
            print("code_check:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("========= entering inner loop ========")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    llama2_new.add_prompt_new("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    llama2_new.add_prompt_new(correct_solution_step_2, mode="answer")
                    if llama2_new.get_prompt()[3:100] == llama2.get_prompt()[3:100]:
                        print("===== Matches inside loop ======")
                        parts = llama2_new.get_prompt().split('[/INST]')
                        desired_text = parts[-1].strip()
                        desired_text = desired_text.strip() 
                        desired_text = desired_text.replace('{', '').replace('}', '')
                        print(desired_text)
                        new_program_list=desired_text.split('\n')
                        print("new_program_list:", new_program_list)
#                         program_list=[]
                        program_list.extend(new_program_list)
                        print("program_list:", program_list)
#                         appended_part_new = llama2_new.get_prompt()[3:][len(llama2.get_prompt())-1:]
                    else:
                        print("no match found")
#                         
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Task: Switch between the tabs to find and click on the link "lorem".
========= Matches outside loop ==========
appended part: action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action) }
program_list: ["action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) }']
HIHI
code_check: action = click_action1('a', '2', observation)
HIHI
code_check: observation, reward, terminated, truncated, info = env.step(action) 
========= entering inner loop ========
===== Matches inside loop ======
 action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action) 
new_program_list: [" action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ']
program_list: ["action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) }', " acti

code_check: observation, reward, terminated, truncated, info = env.step(action) 
========= entering inner loop ========
===== Matches inside loop ======
 action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action) 
new_program_list: [" action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ']
program_list: ["action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) }', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, tru

code_check: observation, reward, terminated, truncated, info = env.step(action) 
========= entering inner loop ========
===== Matches inside loop ======
 action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action) 
new_program_list: [" action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ']
program_list: ["action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) }', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, tru

code_check: observation, reward, terminated, truncated, info = env.step(action) 
========= entering inner loop ========
===== Matches inside loop ======
 action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action) 
new_program_list: [" action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ']
program_list: ["action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) }', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, tru

code_check: observation, reward, terminated, truncated, info = env.step(action) 
========= entering inner loop ========
===== Matches inside loop ======
 action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action) 
new_program_list: [" action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ']
program_list: ["action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) }', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, tru

code_check: observation, reward, terminated, truncated, info = env.step(action) 
========= entering inner loop ========
===== Matches inside loop ======
 action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action) 
new_program_list: [" action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ']
program_list: ["action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) }', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, tru

code_check: observation, reward, terminated, truncated, info = env.step(action) 
========= entering inner loop ========
===== Matches inside loop ======
 action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action) 
new_program_list: [" action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ']
program_list: ["action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) }', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, tru

code_check: observation, reward, terminated, truncated, info = env.step(action) 
========= entering inner loop ========
===== Matches inside loop ======
 action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action) 
new_program_list: [" action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ']
program_list: ["action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) }', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, truncated, info = env.step(action) ', " action = click_action1('a', '2', observation)", 'observation, reward, terminated, tru

In [111]:
count = 0
iteration = 1
incorrect_program_list = []
for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-hard-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
#     print(llama2_new.get_prompt())
    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
        print("========= Matches outside loop ==========")
#     print("appended part:",appended_part)
    program_list = appended_part.split('\n')
    print("program_list:", program_list)
    reward = 0

    for code in program_list:
        new_program_list=[]
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        code_check= code_check.replace('{', '').replace('}', '')
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            exec(cleaned_code)
            print("code_check:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("========= entering inner loop ========")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=1600)
                    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution[3:100] == llama2.get_prompt()[3:100]:
                        print("===== Matches inside loop ======")
                        parts = solution.split('[/INST]')
                        desired_text = parts[-1].strip()
                        desired_text = desired_text.strip() 
                        desired_text = desired_text.replace('{', '').replace('}', '')
                        print(desired_text)
                        new_program_list=desired_text.split('\n')
                        print("new_program_list:", new_program_list)
#                         program_list=[]
                        program_list.extend(new_program_list)
                        print("program_list:", program_list)
#                         appended_part_new = llama2_new.get_prompt()[3:][len(llama2.get_prompt())-1:]
                    else:
                        print("no match found")
#                         (llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "aliquam,".


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


========= Matches outside loop ==========
program_list: ['To solve this task, we can use the `click_action1` function to click on the `div` element with the text "tabs-1", and then use the `enter_text_action` function to type the text "aliquam" into the input field of the `a` element with the text "aliquam,".', 'Here is the code:', '```', "action = click_action1('div', 'tabs-1', observation)", 'observation, reward, terminated, truncated, info = env.step(action)', "action = enter_text_action('aliquam', observation)", 'observation, reward, terminated, truncated, info = env.step(action)', '```', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


no match found
code_check: action = enter_text_action('aliquam', observation)
code_check: observation, reward, terminated, truncated, info = env.step(action)
========= entering inner loop ========
no match found


In [62]:
appended_part_new

's diam\', \'t viverra.\']; Task: Switch between the tabs to find and click on the link "Ultrices". Solution? } [/INST]{ action = click_action1(\'a\', \'1\', observation)\nobservation, reward, terminated, truncated, info = env.step(action) }'

In [57]:
llama2_new.get_prompt()[3:]

'[INST] <<SYS>> { You are designed to generate programs to solve a wide range of complex web interface tasks.\nYou should be able to generate the program using either one or a composition of predefined action functions \nalong with general python codes to solve different tasks. You should not conversate with human in any context.\n \nYou should only use the functions provided herewith in the function description. \nHere is the list for the pre-defined functions [click_action1, enter_text_action, coord_click_action, count_type]\n\nTo use a function, please refer to the Name, Input, Output, Description of the functions, and usage examples below. \nAction functions should be called correctly in the solution. \n\nFunction Name: click_action1\nInput: tag or element, id or text, observation\nOutput: create_click_element_action\nDescription: useful when you want to click on an element in the web interface. This functions cannot be \n            generalized on names. Normally first input is on

In [58]:
llama2.get_prompt()[3:]

'[INST] <<SYS>> { You are designed to generate programs to solve a wide range of complex web interface tasks.\nYou should be able to generate the program using either one or a composition of predefined action functions \nalong with general python codes to solve different tasks. You should not conversate with human in any context.\n \nYou should only use the functions provided herewith in the function description. \nHere is the list for the pre-defined functions [click_action1, enter_text_action, coord_click_action, count_type]\n\nTo use a function, please refer to the Name, Input, Output, Description of the functions, and usage examples below. \nAction functions should be called correctly in the solution. \n\nFunction Name: click_action1\nInput: tag or element, id or text, observation\nOutput: create_click_element_action\nDescription: useful when you want to click on an element in the web interface. This functions cannot be \n            generalized on names. Normally first input is on

In [59]:
if llama2_new.get_prompt()[3:].startswith(llama2.get_prompt()[3:]):
        print("yes")

In [80]:
parts = llama2_new.get_prompt().split('[/INST]')
desired_text = parts[-1].strip()
print(desired_text)

{ action = click_action1('a', '1', observation)
observation, reward, terminated, truncated, info = env.step(action) }


In [73]:
parts = llama2_new.get_prompt().split('[/INST]')

In [74]:
parts

["<s>[INST] <<SYS>> { You are designed to generate programs to solve a wide range of complex web interface tasks.\nYou should be able to generate the program using either one or a composition of predefined action functions \nalong with general python codes to solve different tasks. You should not conversate with human in any context.\n \nYou should only use the functions provided herewith in the function description. \nHere is the list for the pre-defined functions [click_action1, enter_text_action, coord_click_action, count_type]\n\nTo use a function, please refer to the Name, Input, Output, Description of the functions, and usage examples below. \nAction functions should be called correctly in the solution. \n\nFunction Name: click_action1\nInput: tag or element, id or text, observation\nOutput: create_click_element_action\nDescription: useful when you want to click on an element in the web interface. This functions cannot be \n            generalized on names. Normally first input i

In [ ]:
parts = text.split('[/INST]')
desired_text = parts[-1].strip()
print(desired_text)

In [ ]:
last_code_snippet = program_list[-1]

# Click-tab-medium

In [38]:
count = 0
iteration = 7
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-medium-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
    llama2.add_prompt(task_message, mode="user")
    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
    
    program_list = appended_part.split('\n')
    reward = 0

    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
#             print('Invalid program:', code)
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            print("Executing....")
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#                     llama2.restore_prompt()
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=1600)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    print("appended part:", appended_part)
                    new_program_list = appended_part.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "orci,".


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


code_check: action = click_action1('a', 'Tab #1', observation)
Executing....
code_check: observation, reward, terminated, truncated, info = env.step(action)
Executing....
hi
appended part: Here is one possible solution to the task:
import actions
# Click on the first tab
action = click_action1('a', 'Tab #1', observation)
# Observe the change in the environment
observation, reward, terminated, truncated, info = env.step(action)
# Click on the second tab
action = click_action1('a', 'Tab #2', observation)
# Observe the change in the environment
observation, reward, terminated, truncated, info = env.step(action)
# Switch to the first tab
action = click_action1('a', 'Tab #1', observation)
# Find and click on the link "orci,"
link = find_link('orci,', observation)
# Click on the link
action = click_action1('a', link['tag'], observation)
# Observe the change in the environment
observation, reward, terminated, truncated, info = env.step(action)



















































...

Task: Switch between the tabs to find and click on the link "Adipiscing".
code_check: action = click_action1('a', 'Tab #1', observation)
Executing....
code_check: action = click_action1('a', 'Tab #2', observation)
Executing....
code_check: action = enter_text_action('Adipiscing', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
Iteration 5


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "Semper".
code_check: action = click_action1('a', 'Semper', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
code_check: action = enter_text_action('Semper', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
Iteration 6


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "id".
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
code_check: action = enter_text_action('id', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
Success Rate: 0.14285714285714285


In [ ]:
#overall 0.24 (12/50)

# Click tab easy

In [39]:
count = 0
iteration = 11
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-easy-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
    llama2.add_prompt(task_message, mode="user")
    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
    
    program_list = appended_part.split('\n')
    reward = 0

    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
#             print('Invalid program:', code)
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            print("Executing....")
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#                     llama2.restore_prompt()
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=1600)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    print("appended part:", appended_part)
                    new_program_list = appended_part.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "mollis".


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


code_check: action = click_action1('a', 'Tab #1', observation)
Executing....
code_check: observation, reward, terminated, truncated, info = env.step(action)
Executing....
hi


...

Task: Switch between the tabs to find and click on the link "duis".
code_check: action = click_action1('div', 'wrap', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
code_check: action = enter_text_action('duis', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
Iteration 9


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "neque,".
code_check: action = click_action1('div', 'wrap', observation)
Executing....
Output problem: name 'obs' is not defined
code_check: action = click_action1('a', 'Tab #1', observation)
Executing....
code_check: action = enter_text_action('vitae', observation)
Executing....
Output problem: name 'obs' is not defined
code_check: action = click_action1('t', 'arcu elit', observation)
Executing....
code_check: action = enter_text_action('Lectus', observation)
Executing....
Output problem: name 'obs' is not defined
code_check: action = click_action1('span', 'Mi.', observation)
Executing....
code_check: action = enter_text_action('condimentum', observation)
Executing....
Output problem: name 'obs' is not defined
code_check: action = click_action1('tincidunt', 'pretium', observation)
Executing....
Output problem: string indices must be integers
Iteration 10


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "sed".
code_check: action = click_action1('div', 'wrap', observation)
Executing....
code_check: action = click_action1('a', 'Tab #1', observation)
Executing....
code_check: action = enter_text_action('t cursus et', observation)
Executing....
code_check: action = click_action1('div', 'tabs-1', observation)
Executing....
code_check: action = enter_text_action('t aliquam integer', observation)
Executing....
code_check: action = click_action1('p', 't Amet', observation)
Executing....
code_check: action = enter_text_action('t Consequat', observation)
Executing....
code_check: action = click_action1('span', 'semper', observation)
Executing....
code_check: action = enter_text_action('t eget', observation)
Executing....
code_check: action = click_action1('span', 'sed', observation)
Executing....
code_check: action = enter_text_action('t nunc', observation)
Executing....
Success Rate: 0.0


In [40]:
#overall 0+ 0+ 0+ 0

# Click collapsible v1

In [42]:
count = 0
iteration = 11
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-collapsible-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
    
    program_list = appended_part.split('\n')
    reward = 0

    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
#             print('Invalid program:', code)
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            print("Executing....")
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#                     llama2.restore_prompt()
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=1600)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    print("appended part:", appended_part)
                    new_program_list = appended_part.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Expand the section below and click submit.
Output problem: click_action1() missing 1 required positional argument: 'observation'
Output problem: click_action1() missing 1 required positional argument: 'observation'
Iteration 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Expand the section below and click submit.
code_check: action = click_action1('div', 'Section #12', observation)
Executing....
code_check: action = enter_text_action('Expand', observation)
Executing....
code_check: action = click_action1('button', 'Submit', observation)
Executing....
code_check: observation, reward, terminated, truncated, info = env.step(action)
Executing....
Iteration 2


...

Task: Expand the section below and click submit.
code_check: action = click_action1('div', 'Section #18', observation)
Executing....
code_check: action = enter_text_action('Expand', observation)
Executing....
code_check: action = click_action1('button', 'Submit', observation)
Executing....
code_check: observation, reward, terminated, truncated, info = env.step(action)
Executing....
Iteration 9


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Expand the section below and click submit.
Output problem: click_action1() missing 1 required positional argument: 'observation'
Output problem: click_action1() missing 1 required positional argument: 'observation'
Iteration 10


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Expand the section below and click submit.
Output problem: click_action1() missing 1 required positional argument: 'observation'
Output problem: click_action1() missing 1 required positional argument: 'observation'
Success Rate: 0.0


# Click collapsible v2

In [24]:
count = 0
iteration = 16
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-collapsible-2-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
    
    program_list = appended_part.split('\n')
    reward = 0

    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
#             print('Invalid program:', code)
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            print("Executing....")
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#                     llama2.restore_prompt()
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=1600)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    print("appended part:", appended_part)
                    new_program_list = appended_part.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Expand the sections below, to find and click on the link "quis".
code_check: Action: click_action1('div', 'wrap', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
code_check: Action: enter_text_action('quis', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
code_check: Action: click_action1('a', 'quis', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
Iteration 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Expand the sections below, to find and click on the link "tempor".


KeyboardInterrupt: 

In [39]:
### out of 50 all 0

In [16]:

env = gym.make("miniwob/terminal-v1", render_mode="human")
observation, info = env.reset()

# Navigate tree

In [38]:
count = 0
iteration = 4
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/navigate-tree-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
    
    program_list = appended_part.split('\n')
    reward = 0

    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
#             print('Invalid program:', code)
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            print("Executing....")
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#                     llama2.restore_prompt()
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=1600)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    print("appended part:", appended_part)
                    new_program_list = appended_part.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Navigate through the file tree. Find and click on the folder or file named "Bernardine".
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
Iteration 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Navigate through the file tree. Find and click on the folder or file named "Keneth".
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
Iteration 2


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


appended part: To navigate through the file tree and find the folder or file named "Briana", we can use the following actions:
1. Click on the "div wrap" element, as it is the parent element of the other elements in the image.
2. Use the "enter_text_action" function to type the text "folder" into the "input_text" field of the "div area" element. This will select the "folder" option in the drop-down menu.
3. Use the "click_action1" function to click on the "folder" option in the drop-down menu. This will select the "folder" element.
4. Use the "enter_text_action" function to type the text "Briana" into the "input_text" field of the "div area" element. This will select the "Briana" folder.
5. Use the "click_action1" function to click on the "Briana" folder. This will navigate to the "Briana" folder.

Here is the complete solution:
action = click_action1('div wrap', observation)
action = enter_text_action('folder', observation)
action = click_action1('folder', observation)
action = enter_

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


appended part: To navigate through the file tree and find the folder or file named "Briana", we can use the following actions:
1. Click on the "div wrap" element, as it is the parent element of the other elements in the image.
2. Use the "enter_text_action" function to type the text "folder" into the "input_text" field of the "div area" element. This will select the "folder" option in the drop-down menu.
3. Use the "click_action1" function to click on the "folder" option in the drop-down menu. This will select the "folder" element.
4. Use the "enter_text_action" function to type the text "Briana" into the "input_text" field of the "div area" element. This will select the "Briana" folder.
5. Use the "click_action1" function to click on the "Briana" folder. This will navigate to the "Briana" folder.

Here is the complete solution:
action = click_action1('div wrap', observation)
action = enter_text_action('folder', observation)
action = click_action1('folder', observation)
action = enter_

appended part: To navigate through the file tree and find the folder or file named "Briana", we can use the following actions:
1. Click on the "div wrap" element, as it is the parent element of the other elements in the image.
2. Use the "enter_text_action" function to type the text "folder" into the "input_text" field of the "div area" element. This will select the "folder" option in the drop-down menu.
3. Use the "click_action1" function to click on the "folder" option in the drop-down menu. This will select the "folder" element.
4. Use the "enter_text_action" function to type the text "Briana" into the "input_text" field of the "div area" element. This will select the "Briana" folder.
5. Use the "click_action1" function to click on the "Briana" folder. This will navigate to the "Briana" folder.

Here is the complete solution:
action = click_action1('div wrap', observation)
action = enter_text_action('folder', observation)
action = click_action1('folder', observation)
action = enter_

In [39]:
### overall success rate : 0.14 ( 7/50)

# guess number

In [42]:
count = 0
iteration = 9
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/guess-number-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
    
    program_list = appended_part.split('\n')
    reward = 0

    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
#             print('Invalid program:', code)
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            print("Executing....")
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#                     llama2.restore_prompt()
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=1600)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    print("appended part:", appended_part)
                    new_program_list = appended_part.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Guess the number between 0-9 and press Submit. Use the feedback below to find the right number.
Iteration 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Guess the number between 0-9 and press Submit. Use the feedback below to find the right number.
Iteration 2


...

Task: Guess the number between 0-9 and press Submit. Use the feedback below to find the right number.
Iteration 7


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Guess the number between 0-9 and press Submit. Use the feedback below to find the right number.
Iteration 8


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Guess the number between 0-9 and press Submit. Use the feedback below to find the right number.
Success Rate: 0.0


# search engine

In [44]:
count = 0
iteration = 11
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/search-engine-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
    
    program_list = appended_part.split('\n')
    reward = 0

    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
#             print('Invalid program:', code)
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            print("Executing....")
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#                     llama2.restore_prompt()
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=1600)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    print("appended part:", appended_part)
                    new_program_list = appended_part.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Use the textbox to enter "Lyda" and press "Search", then find and click the 3rd search result.
Iteration 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Use the textbox to enter "Riley" and press "Search", then find and click the 3rd search result.
code_check: Action: click_action1('div wrap', 'search-bar', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
code_check: Action: click_action1('button Search', 'Search', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
Output problem: name 'obs' is not defined
Output problem: unexpected indent (<string>, line 1)
code_check: Action: click_action1('div', 'third search result', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
Iteration 2


...

Task: Use the textbox to enter "Ashlea" and press "Search", then find and click the 4th search result.
code_check: action = enter_text_action('Ashlea', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
code_check: action = enter_text_action('Ashlea', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
Iteration 9


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Use the textbox to enter "Juan" and press "Search", then find and click the 2nd search result.
Output problem: unexpected indent (<string>, line 1)
Iteration 10


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Use the textbox to enter "Keli" and press "Search", then find and click the 7th search result.
Success Rate: 0.0


In [51]:
## overall == 0

# terminal

In [22]:
env = gym.make("miniwob/terminal-v1", render_mode="human")
observation, info = env.reset()
task = observation["utterance"]

In [24]:
objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
objects_in_the_image

['div wrap',
 'div area',
 'div ',
 'div terminal',
 'div terminal',
 'div terminal-contents',
 'div ',
 'span Welcome! Type help for a list of available commands.',
 'div ',
 'span Last login: Sat Dec 16 2023',
 'div ',
 'span user$',
 'span _',
 'input_text terminal-target']

In [27]:
example="""since we dont have enter command, 
while you are enterting a text using enter_text_action(), use it like this: 
enter_text_action("text_you_want_enter \n", observation)
the usage of "\n" will act like pressing enter after pressing the command
"""

In [33]:
count = 0
iteration = 5
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/terminal-v1")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
#     llama2.add_prompt(example, mode="user")
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=4000)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
    
    program_list = appended_part.split('\n')
    reward = 0

    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
#             print('Invalid program:', code)
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            print("Executing....")
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#                     llama2.restore_prompt()
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=4000)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    print("appended part:", appended_part)
                    new_program_list = appended_part.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.ap render_mode="human"pend([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

SyntaxError: invalid syntax (51056944.py, line 62)

In [50]:
### overll == 0 + 1+2+0+

# email

In [48]:
count = 0
iteration = 9
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/email-inbox-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
    
    program_list = appended_part.split('\n')
    reward = 0

    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
#             print('Invalid program:', code)
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            print("Executing....")
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#                     llama2.restore_prompt()
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=1600)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    print("appended part:", appended_part)
                    new_program_list = appended_part.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Find the email by Sileas and click the star icon to mark it as important.
Error in generated expression  observation, reward, terminated,
Iteration 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Find the email by Olva and forward that email to Freda.
Iteration 2


...

Task: Find the email by Germana and reply to them with the text "Orci. Risus, nisi praesent.".
Output problem: unexpected indent (<string>, line 1)
code_check: Action: enter_text_action('email', observation)
Executing....
Output problem: unexpected indent (<string>, line 1)
Output problem: unexpected indent (<string>, line 1)
Iteration 7


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Find the email by Gwennie and forward that email to Prudi.
Iteration 8


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Find the email by Robinet and click the trash icon to delete it.
Success Rate: 0.0


In [32]:
count = 0
iteration = 50
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-pie-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=1600)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    if solution.startswith(llama2.get_prompt()[3:]):
        appended_part = solution[len(llama2.get_prompt())-1:]
    
    program_list = appended_part.split('\n')
    reward = 0

    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
#             print('Invalid program:', code)
            continue

        try:
            exec(code_check)
            print("code_check:",code_check)
            print("Executing....")
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
#                     llama2.restore_prompt()
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=1600)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    print("appended part:", appended_part)
                    new_program_list = appended_part.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Expand the pie menu below and click on the item labeled "Y".
Iteration 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Expand the pie menu below and click on the item labeled "1".
Output problem: unexpected indent (<string>, line 1)
Iteration 2


...

appended part: To expand the pie menu and click on the item labeled "0", we can use the following actions:
1. Click on the "div wrap" element using the `click_action1` function.
2. Type "0" into the input field of the "div area" element using the `enter_text_action` function.
3. Click on the "div divWheel" element using the `click_action1` function.
4. Observe the changes in the objects in the image after performing the above actions. If the "0" item is now clickable, we can click on it using the `click_action1` function.
Here is the complete solution:
action = click_action1('div wrap')
action = enter_text_action('0', observation)
action = click_action1('div divWheel')
observation, reward, terminated, truncated, info = env.step(action)





























































































code_check: action = enter_text_action('0', observation)
Executing....
code_check: observation, reward, terminated, truncated, info = env.step(action)
Executing.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


appended part: To expand the pie menu and click on the item labeled "0", we can use the following actions:
1. Click on the "div wrap" element using the `click_action1` function.
2. Type "0" into the input field of the "div area" element using the `enter_text_action` function.
3. Click on the "div divWheel" element using the `click_action1` function.
4. Observe the changes in the objects in the image after performing the above actions. If the "0" item is now clickable, we can click on it using the `click_action1` function.
Here is the complete solution:
action = click_action1('div wrap')
action = enter_text_action('0', observation)
action = click_action1('div divWheel')
observation, reward, terminated, truncated, info = env.step(action)





























































































code_check: action = enter_text_action('0', observation)
Executing....
code_check: observation, reward, terminated, truncated, info = env.step(action)
Executing.

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


appended part: To expand the pie menu and click on the item labeled "0", we can use the following actions:
1. Click on the "div wrap" element using the `click_action1` function.
2. Type "0" into the input field of the "div area" element using the `enter_text_action` function.
3. Click on the "div divWheel" element using the `click_action1` function.
4. Observe the changes in the objects in the image after performing the above actions. If the "0" item is now clickable, we can click on it using the `click_action1` function.
Here is the complete solution:
action = click_action1('div wrap')
action = enter_text_action('0', observation)
action = click_action1('div divWheel')
observation, reward, terminated, truncated, info = env.step(action)





























































































code_check: action = enter_text_action('0', observation)
Executing....
code_check: observation, reward, terminated, truncated, info = env.step(action)
Executing.

# Click tab 2 hard- one shot

In [37]:
task_message = """This task is a multi-step task, directly performing a series of actions may not solve the task.
Need to observe the changes in the objects in image after performing an/any action to see if 
further actions need to be made to solve the task or not. """

In [21]:
example="""
Here is one multi-step task example, this task can only be solved step-by-step. You should wait until new content has been shown and perform 1 action at a time

# Initially at tab 1

Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 't Elit', 'span leo'];
Task: 'Switch between the tabs to find and click on the link "ipsum".'
# Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;
Solutions?

# Swtich between tabs

action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action)

Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span b', 'span sed'];
Task: 'Switch between the tabs to find and click on the link "ipsum".;
# Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;
Solutions?

# Swtich between tabs

action = click_action1('a', '3', observation)
observation, reward, terminated, truncated, info = env.step(action)

Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 't ipsum', 'span c', 'span d'];
Task: 'Switch between the tabs to find and click on the link "ipsum".;
# Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;
Solutions?

# Swtich between tabs

action = click_action1('a', '4', observation)
observation, reward, terminated, truncated, info = env.step(action)

Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span e', 'span f'];
Task: 'Switch between the tabs to find and click on the link "ipsum".;
# Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;
Solutions?

# Swtich between tabs

action = click_action1('a', '5', observation)
observation, reward, terminated, truncated, info = env.step(action)

Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span g', 'span h'];
Task: 'Switch between the tabs to find and click on the link "ipsum".;
# Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;
Solutions?

# Swtich between tabs

action = click_action1('a', '6', observation)
observation, reward, terminated, truncated, info = env.step(action)

Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span f', 'span ipsum'];
Task: 'Switch between the tabs to find and click on the link "ipsum".;
# Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;
Solutions?

# Now you have found the target link

action = click_action1('span', 'ipsum', observation)
observation, reward, terminated, truncated, info = env.step(action)

Task has been successfully solved, note that since previous does not have what we want, so we had to search all the three sections, otherwise if what we want is found in earlier sections, we can stop earlier!!!
"""

In [29]:
one='Here is an example of a multi-step task, this task can only be solved step-by-step. You should wait until new content has been shown and perform 1 action at a time'

two= '# Initially at tab 1'

three = '''Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 't Elit', 'span leo'];
             Task: 'Switch between the tabs to find and click on the link "ipsum".'
             # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed.  Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;\
             Solutions?'''
four='''# Swtich between tabs'''

five='''action = click_action1('a', '2', observation)
observation, reward, terminated, truncated, info = env.step(action)'''

six = '''Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span b', 'span sed'];
             Task: 'Switch between the tabs to find and click on the link "ipsum".;
             # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed.  Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;\
             Solutions?'''
seven = '''# Swtich between tabs'''

eight = '''action = click_action1('a', '3', observation)
observation, reward, terminated, truncated, info = env.step(action)'''

nine = '''Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 't ipsum', 'span c', 'span d'];
             Task: 'Switch between the tabs to find and click on the link "ipsum".;
             # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed.  Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;\
             Solutions?'''

ten = '''# Swtich between tabs'''

eleven= '''action = click_action1('a', '4', observation)
observation, reward, terminated, truncated, info = env.step(action)'''

twelve= '''Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span e', 'span f'];
             Task: 'Switch between the tabs to find and click on the link "ipsum".;
             # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed.  Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;\
             Solutions?'''

thirteen =   '''# Swtich between tabs'''
    
fourteen = '''action = click_action1('a', '5', observation)
observation, reward, terminated, truncated, info = env.step(action)'''

fifteen = '''Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span g', 'span h'];
             Task: 'Switch between the tabs to find and click on the link "ipsum".;
             # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed.  Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;\
             Solutions?'''

sixteen = '''# Swtich between tabs'''

seventeen= '''action = click_action1('a', '6', observation)
observation, reward, terminated, truncated, info = env.step(action)'''

eighteen = '''Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span f', 'span ipsum'];
             Task: 'Switch between the tabs to find and click on the link "ipsum".;
             # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed.  Remeber, you cannot fake the user feedback but you need wait to recevie from user!!;\
             Solutions?'''

nineteen = '''# Now you have found the link'''

twenty = '''action = click_action1('span', 'ipsum', observation)
observation, reward, terminated, truncated, info = env.step(action)'''

twentyone = 'Task has been successfully solved, note that since previous does not have what we want, so we had to search all the three sections, otherwise if what we want is found in earlier sections, we can stop earlier!!!'


In [31]:
count = 0
iteration = 5
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-hard-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
    llama2.add_prompt(one, mode= 'user')
    llama2.add_prompt(two, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(three, mode= 'user')
    llama2.add_prompt(four, mode= 'answer')
    llama2.add_prompt(five, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(six, mode= 'user')
    llama2.add_prompt(seven, mode= 'answer')
    llama2.add_prompt(eight, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(nine, mode= 'user')
    llama2.add_prompt(ten, mode= 'answer')
    llama2.add_prompt(eleven, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(twelve, mode= 'user')
    llama2.add_prompt(thirteen, mode= 'answer')
    llama2.add_prompt(fourteen, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(fifteen, mode= 'user')
    llama2.add_prompt(sixteen, mode= 'answer')
    llama2.add_prompt(seventeen, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(eighteen, mode= 'user')
    llama2.add_prompt(nineteen, mode= 'answer')
    llama2.add_prompt(twenty, mode= 'answer')
    llama2.add_prompt(twentyone, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=4000)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    parts = solution.split('[/INST]')
    desired_text = parts[-1].strip()
    desired_text = desired_text.strip() 
    if solution.startswith(llama2.get_prompt()[3:]):
        print("HIHIHI")
#         appended_part = solution[len(llama2.get_prompt())-1:]
#         print("appended part: ----",appended_part)
    program_list = desired_text.split('\n')
    reward = 0
    
    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            exec(cleaned_code)
            print(".........Executing.......")
            print("code_check:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=4000)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    parts = new_solution.split('[/INST]')
                    desired_text = parts[-1].strip()
                    desired_text = desired_text.strip() 
                    new_program_list = desired_text.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "Fusce".


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


HIHIHI
click_action1
.........Executing.......
code_check: action = click_action1('div', 'wrap', observation)
.........Executing.......
code_check: observation, reward, terminated, truncated, info = env.step(action)
hi


...

Task: Switch between the tabs to find and click on the link "elementum".
HIHIHI
Iteration 3


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "bibendum".
HIHIHI
Iteration 4


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "felis.".
HIHIHI
Success Rate: 0.0


In [ ]:
4/10+ 2/5+0/5

# navigate tree

In [34]:
one = "'Here is a multi-step task example, this task can only be solved step-by-step. You should wait until new content has been shown and perform 1 action at a time.'"

two = " 'Objects in Image: ['ul tree', 'folder Marcella', 'folder Enola', 'folder Agustina', 'folder Ashlea', 'span Cristin']; Task: 'Navigate through the file tree. Find and click on the folder or file named "Keli". # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remember, you cannot fake the user feedback but you need to wait to receive from user!! Solutions?'"

three = " 'action = click_action1('folder', 'Marcella', observation) observation, reward, terminated, truncated, info = env.step(action)'"

four = "'Objects in Image: ['ul tree', 'folder Marcella','ul ','span kasie','folder Enola', 'folder Agustina', 'folder Ashlea', 'span Cristin']; Task: 'Navigate through the file tree. Find and click on the folder or file named "Keli". # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remember, you cannot fake the user feedback but you need to wait to receive from user!! Solutions?'"

five = " 'action = click_action1('folder', 'Enola', observation) observation, reward, terminated, truncated, info = env.step(action)'"

six - "'Objects in Image: ['ul tree', 'folder Marcella','ul ','span kasie','folder Enola', 'ul ', 'folder Cierra', 'folder Agustina', 'folder Ashlea', 'span Cristin']; Task: 'Navigate through the file tree. Find and click on the folder or file named "Keli". # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remember, you cannot fake the user feedback but you need to wait to receive from user!! Solutions?'"

seven - " 'action = click_action1('folder', 'Agustina', observation) observation, reward, terminated, truncated, info = env.step(action)'"

eight - "'Objects in Image: ['ul tree', 'folder Marcella','ul ','span kasie','folder Enola', 'ul ', 'folder Cierra', 'folder Agustina', 'folder Ashlea', 'span Cristin']; Task: 'Navigate through the file tree. Find and click on the folder or file named "Keli". # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remember, you cannot fake the user feedback but you need to wait to receive from user!! Solutions?'"

nine - " 'action = click_action1('folder', 'Ashlea', observation) observation, reward, terminated, truncated, info = env.step(action)'"

ten - " 'Objects in Image: ['ul tree', 'folder Marcella','ul ','span kasie','folder Enola', 'ul ', 'folder Cierra', 'folder Agustina', 'folder Ashlea', 'ul ', 'folder Keli', 'span Renda', 'span Cristin']; Task: 'Navigate through the file tree. Find and click on the folder or file named "Keli". # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remember, you cannot fake the user feedback but you need to wait to receive from user!! Solutions?'"

eleven - " '# action = click_action1('folder', 'Keli', observation) observation, reward, terminated, truncated, info = env.step(action)'"

twelve - " 'Task has been successfully solved, note that since section 1 and section 2 does not have what we want, so we had to search all the three sections, otherwise if what we want is found in earlier sections, we can stop earlier!!!'"

In [38]:
count = 0
iteration = 20
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/navigate-tree-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
#     llama2.add_prompt(example, "user")
    llama2.add_prompt(one, mode= 'user')
    llama2.add_prompt(two, mode= 'user')
    llama2.add_prompt(three, mode= 'user')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(four, mode= 'user')
    llama2.add_prompt(five, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(six, mode= 'user')
    llama2.add_prompt(seven, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(eight, mode= 'user')
    llama2.add_prompt(nine, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(ten, mode= 'user')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(eleven, mode= 'answer')
    llama2.add_prompt(twelve, mode= 'user')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    
    
#     llama2.add_prompt(thirteen, mode= 'answer')
#     llama2.add_prompt(fourteen, mode= 'answer')
#     
#     llama2.add_prompt(fifteen, mode= 'user')
#     llama2.add_prompt(sixteen, mode= 'answer')
#     llama2.add_prompt(seventeen, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(eighteen, mode= 'user')
#     llama2.add_prompt(nineteen, mode= 'answer')
#     llama2.add_prompt(twenty, mode= 'answer')
#     llama2.add_prompt(twentyone, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
    
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=4000)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    parts = solution.split('[/INST]')
    desired_text = parts[-1].strip()
    desired_text = desired_text.strip() 
    if solution.startswith(llama2.get_prompt()[3:]):
        print("HIHIHI")
#         appended_part = solution[len(llama2.get_prompt())-1:]
#         print("appended part: ----",appended_part)
    program_list = desired_text.split('\n')
    reward = 0
    
    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            exec(cleaned_code)
            print(".........Executing.......")
            print("code_check:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=4000)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    parts = new_solution.split('[/INST]')
                    desired_text = parts[-1].strip()
                    desired_text = desired_text.strip() 
                    new_program_list = desired_text.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Navigate through the file tree. Find and click on the folder or file named "Keneth".
HIHIHI
Iteration 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Navigate through the file tree. Find and click on the folder or file named "Juan".
HIHIHI
click_action1
enter_text_action
count_type
click_action1
Output problem: invalid syntax (<string>, line 1)
Iteration 2


...

Task: Navigate through the file tree. Find and click on the folder or file named "Augus".
HIHIHI
Output problem: click_action1() missing 1 required positional argument: 'observation'
Output problem: click_action1() missing 1 required positional argument: 'observation'
Output problem: click_action1() missing 1 required positional argument: 'observation'
Output problem: click_action1() missing 1 required positional argument: 'observation'
Output problem: click_action1() missing 1 required positional argument: 'observation'
Iteration 18


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Navigate through the file tree. Find and click on the folder or file named "Emile".
HIHIHI
Output problem: click_action1() missing 1 required positional argument: 'observation'
Output problem: click_action1() missing 1 required positional argument: 'observation'
Output problem: click_action1() missing 1 required positional argument: 'observation'
Output problem: click_action1() missing 1 required positional argument: 'observation'
Output problem: click_action1() missing 1 required positional argument: 'observation'
Iteration 19


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Navigate through the file tree. Find and click on the folder or file named "Chas".
HIHIHI
Output problem: click_action1() missing 1 required positional argument: 'observation'
env.step
Output problem: click_action1() missing 1 required positional argument: 'observation'
env.step
Output problem: click_action1() missing 1 required positional argument: 'observation'
env.step
Output problem: click_action1() missing 1 required positional argument: 'observation'
env.step
Output problem: click_action1() missing 1 required positional argument: 'observation'
env.step
.........Executing.......
code_check: observation, reward, terminated, truncated, info = env.step(enter_text_action('Chas', observation))
env.step
Output problem: click_action1() missing 1 required positional argument: 'observation'
env.step
Output problem: click_action1() missing 1 required positional argument: 'observation'
Success Rate: 0.0


In [ ]:
40 === 9/20 + 0/20

# guess number

In [36]:
one - "'Here is one multi-step task example, this task can only be solved step-by-step. You should wait until new content has been shown.'"

two - "'Objects in Image: ['input_number tt', 'button Submit']; Task: 'Guess the number between 0-9 and press Submit. Use the feedback below to find the right number.' Solutions?'"

three - " 'action = click_action1('input_number', 'tt', observation) observation, reward, terminated, truncated, info = env.step(action) action = enter_text_action('5', observation) observation, reward, terminated, truncated, info = env.step(action) action = click_action1('button', 'Submit', observation) observation, reward, terminated, truncated, info = env.step(action)'"

four - " 'The number is lower than 5.'"

five - "Assistant: 'action = click_action1('input_number', 'tt', observation) observation, reward, terminated, truncated, info = env.step(action) action = enter_text_action(backspace, observation) observation, reward, terminated, truncated, info = env.step(action) action = enter_text_action('3', observation) observation, reward, terminated, truncated, info = env.step(action) action = click_action1('button', 'Submit', observation) observation, reward, terminated, truncated, info = env.step(action)'"

six - "User: 'The number is higher than 3.'"

seven - "Assistant: 'action = click_action1('input_number', 'tt', observation) observation, reward, terminated, truncated, info = env.step(action) action = enter_text_action(backspace, observation) observation, reward, terminated, truncated, info = env.step(action) action = enter_text_action('4', observation) observation, reward, terminated, truncated, info = env.step(action) action = click_action1('button', 'Submit', observation) observation, reward, terminated, truncated, info = env.step(action)'"

eight - "Assistant: 'Task has been successfully solved, note that since section 1 and section 2 does not have what we want, so we had to search all the three sections, otherwise if what we want is found in earlier sections, we can stop earlier!!!'"

In [39]:
count = 0
iteration = 10
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-hard-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
#     llama2.add_prompt(example, "user")
    llama2.add_prompt(one, mode= 'user')
    llama2.add_prompt(two, mode= 'user')
    llama2.add_prompt(three, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(four, mode= 'user')
    llama2.add_prompt(five, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(six, mode= 'user')
    llama2.add_prompt(seven, mode= 'answer')
    llama2.add_prompt(eight, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(nine, mode= 'user')
#     llama2.add_prompt(ten, mode= 'answer')
#     llama2.add_prompt(eleven, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(twelve, mode= 'user')
#     llama2.add_prompt(thirteen, mode= 'answer')
#     llama2.add_prompt(fourteen, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(fifteen, mode= 'user')
#     llama2.add_prompt(sixteen, mode= 'answer')
#     llama2.add_prompt(seventeen, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(eighteen, mode= 'user')
#     llama2.add_prompt(nineteen, mode= 'answer')
#     llama2.add_prompt(twenty, mode= 'answer')
#     llama2.add_prompt(twentyone, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
    
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=4000)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    parts = solution.split('[/INST]')
    desired_text = parts[-1].strip()
    desired_text = desired_text.strip() 
    if solution.startswith(llama2.get_prompt()[3:]):
        print("HIHIHI")
#         appended_part = solution[len(llama2.get_prompt())-1:]
#         print("appended part: ----",appended_part)
    program_list = desired_text.split('\n')
    reward = 0
    
    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            exec(cleaned_code)
            print(".........Executing.......")
            print("code_check:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=4000)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    parts = new_solution.split('[/INST]')
                    desired_text = parts[-1].strip()
                    desired_text = desired_text.strip() 
                    new_program_list = desired_text.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "massa".


HIHIHI
.........Executing.......
code_check: observation, reward, terminated, truncated, info = env.step(click_action1('a 2', 'massa', observation))
Iteration 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Task: Switch between the tabs to find and click on the link "Enim,".
HIHIHI
Iteration 8


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "Facilisis".
HIHIHI
Iteration 9


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "iaculis.".
HIHIHI
Success Rate: 0.0


In [ ]:
0/20 , 0/10

In [28]:
solution

'[INST] <<SYS>> { You are designed to generate programs to solve a wide range of complex web interface tasks.\nYou should be able to generate the program using either one or a composition of predefined action functions \nalong with general python codes to solve different tasks. You should not conversate with human in any context.\n \nYou should only use the functions provided herewith in the function description. \nHere is the list for the pre-defined functions [click_action1, enter_text_action, coord_click_action, count_type]\n\nTo use a function, please refer to the Name, Input, Output, Description of the functions, and usage examples below. \nAction functions should be called correctly in the solution. \n\nFunction Name: click_action1\nInput: tag or element, id or text, observation\nOutput: create_click_element_action\nDescription: useful when you want to click on an element in the web interface. This functions cannot be \n            generalized on names. Normally first input is on

In [ ]:
llama2.get_prompt()[3:]

In [24]:
desired_text

"To solve this task, we need to observe the changes in the objects in the image after performing an action. Based on the objects in the image, we can try the following actions:\n1. Click on the first tab 'div wrap' to see if it will change or not.\n observation, reward, terminated, truncated, info = env.step(click_action1('div wrap', observation))\nIf the tab changes, we can try to click on the next tab 'div area' and observe the change. If the tab does not change, we can try to click on the link 'a 1' and observe the change.\n\nPlease provide the updated observation after each action, and we can continue to solve the task."

In [55]:
one = " 'Here is one multi-step task example, this task can only be solved step-by-step. You should wait until new content has been shown.'"

two = " 'Objects in Image: ['input_number tt', 'button Submit']; Task: 'Guess the number between 0-9 and press Submit. Use the feedback below to find the right number.' Solutions?'"

three = " 'action = click_action1('input_number', 'tt', observation) observation, reward, terminated, truncated, info = env.step(action) action = enter_text_action('5', observation) observation, reward, terminated, truncated, info = env.step(action) action = click_action1('button', 'Submit', observation) observation, reward, terminated, truncated, info = env.step(action)'"

four = " 'The number is lower than 5.'"

five = " 'action = click_action1('input_number', 'tt', observation) observation, reward, terminated, truncated, info = env.step(action) action = enter_text_action(backspace, observation) observation, reward, terminated, truncated, info = env.step(action) action = enter_text_action('3', observation) observation, reward, terminated, truncated, info = env.step(action) action = click_action1('button', 'Submit', observation) observation, reward, terminated, truncated, info = env.step(action)'"

six = "'The number is higher than 3.'"

seven = " 'action = click_action1('input_number', 'tt', observation) observation, reward, terminated, truncated, info = env.step(action) action = enter_text_action(backspace, observation) observation, reward, terminated, truncated, info = env.step(action) action = enter_text_action('4', observation) observation, reward, terminated, truncated, info = env.step(action) action = click_action1('button', 'Submit', observation) observation, reward, terminated, truncated, info = env.step(action)'"

eight = " 'Task has been successfully solved, note that since section 1 and section 2 does not have what we want, so we had to search all the three sections, otherwise if what we want is found in earlier sections, we can stop earlier!!!'"


In [56]:
count = 0
iteration = 10
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/guess-number-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
#     llama2.add_prompt(example, "user")
    llama2.add_prompt(one, mode= 'user')
    llama2.add_prompt(two, mode= 'user')
    llama2.add_prompt(three, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(four, mode= 'user')
    llama2.add_prompt(five, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(six, mode= 'user')
    llama2.add_prompt(seven, mode= 'answer')
    llama2.add_prompt(eight, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(nine, mode= 'user')
#     llama2.add_prompt(ten, mode= 'answer')
#     llama2.add_prompt(eleven, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(twelve, mode= 'user')
#     llama2.add_prompt(thirteen, mode= 'answer')
#     llama2.add_prompt(fourteen, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(fifteen, mode= 'user')
#     llama2.add_prompt(sixteen, mode= 'answer')
#     llama2.add_prompt(seventeen, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(eighteen, mode= 'user')
#     llama2.add_prompt(nineteen, mode= 'answer')
#     llama2.add_prompt(twenty, mode= 'answer')
#     llama2.add_prompt(twentyone, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
    
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=4000)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    parts = solution.split('[/INST]')
    desired_text = parts[-1].strip()
    desired_text = desired_text.strip() 
    if solution.startswith(llama2.get_prompt()[3:]):
        print("HIHIHI")
#         appended_part = solution[len(llama2.get_prompt())-1:]
#         print("appended part: ----",appended_part)
    program_list = desired_text.split('\n')
    reward = 0
    
    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            exec(cleaned_code)
            print(".........Executing.......")
            print("code_check:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=4000)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    parts = new_solution.split('[/INST]')
                    desired_text = parts[-1].strip()
                    desired_text = desired_text.strip() 
                    new_program_list = desired_text.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Guess the number between 0-9 and press Submit. Use the feedback below to find the right number.


KeyboardInterrupt: 

In [ ]:
0

In [43]:
one = "'Here is one multi-step task example, this task can only be solved step-by-step. You should wait until new content has been shown and perform 1 action at a time.'"

two ="'# Initial at tab 1'"

three =" 'Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 't Elit', 'span leo']; Task: 'Switch between the tabs to find and click on the link "ipsum". # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remember, you cannot fake the user feedback but you need wait to receive from user!! Solutions?'"

four = " '# Switch between tabs'"

five = " 'action = click_action1('a', '2', observation) observation, reward, terminated, truncated, info = env.step(action)'"

six = " 'Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span b', 'span sed']; Task: 'Switch between the tabs to find and click on the link "ipsum". # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remember, you cannot fake the user feedback but you need wait to receive from user!! Solutions?'"

seven = " '# Switch between tabs'"

eight = " 'action = click_action1('a', '3', observation) observation, reward, terminated, truncated, info = env.step(action)'"

nine = " 'Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 't ipsum', 'span c', 'span d']; Task: 'Switch between the tabs to find and click on the link "ipsum". # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remember, you cannot fake the user feedback but you need wait to receive from user!! Solutions?'"

ten = " '# Switch between tabs'"

eleven = " 'action = click_action1('a', '4', observation) observation, reward, terminated, truncated, info = env.step(action)'"

twelve = " 'Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span e', 'span f']; Task: 'Switch between the tabs to find and click on the link "ipsum". # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remember, you cannot fake the user feedback but you need wait to receive from user!! Solutions?'"

thirteen = "'# Switch between tabs'"

fourteen = " 'action = click_action1('a', '5', observation) observation, reward, terminated, truncated, info = env.step(action)'"

fifteen = "'Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span g', 'span h']; Task: 'Switch between the tabs to find and click on the link "ipsum". # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remember, you cannot fake the user feedback but you need wait to receive from user!! Solutions?'"

sixteen = "'# Switch between tabs'"

seventeen = " 'action = click_action1('a', '6', observation) observation, reward, terminated, truncated, info = env.step(action)'"

eighteen = "'Objects in Image: ['a 1', 'a 2', 'a 3', 'a 4', 'a 5', 'a 6', 'span f', 'span ipsum']; Task: 'Switch between the tabs to find and click on the link "ipsum". # Ask a question after you generate each every action and wait for user input feedback. Also most the important thing is do not generate the same action again and ensure to try new action if the previous did not succeed. Remember, you cannot fake the user feedback but you need wait to receive from user!! Solutions?'"

nineteen = " '# Now you have found the link'"

twenty = " 'action = click_action1('span', 'ipsum', observation) observation, reward, terminated, truncated, info = env.step(action)'"

twenty-one = "'Task has been successfully solved, note that since section 1 and section 2 does not have what we want, so we had to search all the three sections, otherwise if what we want is found in earlier sections, we can stop earlier!!!'"


'''

In [42]:
count = 0
iteration = 10
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/search-engine-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
#     llama2.add_prompt(example, "user")
    llama2.add_prompt(one, mode= 'user')
    llama2.add_prompt(two, mode= 'user')
    llama2.add_prompt(three, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(four, mode= 'user')
    llama2.add_prompt(five, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(six, mode= 'user')
    llama2.add_prompt(seven, mode= 'answer')
    llama2.add_prompt(eight, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(nine, mode= 'user')
    llama2.add_prompt(ten, mode= 'answer')
    llama2.add_prompt(eleven, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(twelve, mode= 'user')
    llama2.add_prompt(thirteen, mode= 'answer')
    llama2.add_prompt(fourteen, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(fifteen, mode= 'user')
    llama2.add_prompt(sixteen, mode= 'answer')
    llama2.add_prompt(seventeen, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(eighteen, mode= 'user')
    llama2.add_prompt(nineteen, mode= 'answer')
    llama2.add_prompt(twenty, mode= 'answer')
    llama2.add_prompt(twentyone, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=4000)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    parts = solution.split('[/INST]')
    desired_text = parts[-1].strip()
    desired_text = desired_text.strip() 
    if solution.startswith(llama2.get_prompt()[3:]):
        print("HIHIHI")
#         appended_part = solution[len(llama2.get_prompt())-1:]
#         print("appended part: ----",appended_part)
    program_list = desired_text.split('\n')
    reward = 0
    
    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            exec(cleaned_code)
            print(".........Executing.......")
            print("code_check:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=4000)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    parts = new_solution.split('[/INST]')
                    desired_text = parts[-1].strip()
                    desired_text = desired_text.strip() 
                    new_program_list = desired_text.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Use the textbox to enter "Vanda" and press "Search", then find and click the 7th search result.
HIHIHI
Iteration 1


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Use the textbox to enter "Renda" and press "Search", then find and click the 6th search result.


...

Task: Use the textbox to enter "Chas" and press "Search", then find and click the 9th search result.


HIHIHI
.........Executing.......
code_check: observation, reward, terminated, truncated, info = env.step(action)
Iteration 9


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Use the textbox to enter "Jess" and press "Search", then find and click the 3rd search result.


HIHIHI
click_action1
.........Executing.......
code_check: action = click_action1('div', 'wrap', observation)
enter_text_action
click_action1
.........Executing.......
code_check: action = click_action1('button', 'Search', observation)
.........Executing.......
code_check: observation, reward, terminated, truncated, info = env.step(action)
Success Rate: 0.0


In [ ]:
4/20+1/10+0/10

In [48]:
zero = "'Here is one multi-step task example, this task can only be solved step-by-step. You should wait until new content has been shown and perform 1 action at a time.'"
one= '''Objects in Image: ['h3 Section #1', 'h3 Section #2', 'h3 Section #3'];
             Task: 'Expand the sections below, to find and click on the link "lacus".';
             Solutions?'''
two = '''action = click_action1('h3', 'Section #1', observation)
observation, reward, terminated, truncated, info = env.step(action)'''

four= '''Objects in Image: ['h3 Section #1', 'span a', 'span b', 'h3 Section #2', 'h3 Section #3'];
             Task: 'Expand the sections below, to find andt click on the link "lacus".';
             Solutions?'''

five= '''action = click_action1('h3', 'Section #2', observation)
observation, reward, terminated, truncated, info = env.step(action)'''

six= '''Objects in Image: ['h3 Section #1','h3 Section #2', 'span lacus sit', 'h3 Section #3'];
             Task: 'Expand the sections below, to find and click on the link "lacus".';
             Solutions?'''

seven= '''action = click_action1('h3', 'Section #3', observation)
observation, reward, terminated, truncated, info = env.step(action)'''

eight= '''Objects in Image: ['h3 Section #1','h3 Section #2', 'h3 Section #3', 'span d', 'span lacus', 't span.'];
             Task: 'Expand the sections below, to find andt click on the link "lacus".';
             Solutions?'''

nine= '''action = click_action1('span', 'lacus', observation)
observation, reward, terminated, truncated, info = env.step(action)'''

In [49]:
count = 0
iteration = 10
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-collapsible-2-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")

    llama2.add_prompt(zero, mode= 'user')
    llama2.add_prompt(one, mode= 'user')
    llama2.add_prompt(two, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(four, mode= 'user')
    llama2.add_prompt(five, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(six, mode= 'user')
    llama2.add_prompt(seven, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(eight, mode= 'user')
    llama2.add_prompt(nine, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')

    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=4000)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    parts = solution.split('[/INST]')
    desired_text = parts[-1].strip()
    desired_text = desired_text.strip() 
    if solution.startswith(llama2.get_prompt()[3:]):
        print("HIHIHI")
#         appended_part = solution[len(llama2.get_prompt())-1:]
#         print("appended part: ----",appended_part)
    program_list = desired_text.split('\n')
    reward = 0
    
    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            exec(cleaned_code)
            print(".........Executing.......")
            print("code_check:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=4000)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    parts = new_solution.split('[/INST]')
                    desired_text = parts[-1].strip()
                    desired_text = desired_text.strip() 
                    new_program_list = desired_text.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Expand the sections below, to find and click on the link "in".


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


.........Executing.......
code_check: observation, reward, terminated, truncated, info = env.step(action)
hi
Iteration 1


...

Task: Expand the sections below, to find and click on the link "purus".


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


click_action1
.........Executing.......
code_check: action = click_action1('h3', 'Section #2', observation)
.........Executing.......
code_check: observation, reward, terminated, truncated, info = env.step(action)
hi
click_action1
.........Executing.......
code_check: action = click_action1('div', 'purus', observation)
Output problem: string indices must be integers
Iteration 9


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Expand the sections below, to find and click on the link "Nec.".
click_action1
.........Executing.......
code_check: action = click_action1('h3', 'Nec.', observation)
Output problem: string indices must be integers
Success Rate: 0.1


In [ ]:
4/10 +6/10 +1/10

In [52]:
one= '''Objects in Image: ['a Tab #1', 'a Tab #2', 'a Tab #3', 't Elit', 'span leo'];
             Task: 'Switch between the tabs to find and click on the link "ipsum".' 
             Solutions?'''
two= '''action = click_action1('a', 'Tab #2', observation)
observation, reward, terminated, truncated, info = env.step(action)'''
three=  '''Objects in Image: ['a Tab #1', 'a Tab #2', 'a Tab #3', 'span b', 'span sed'];
             Task: 'Switch between the tabs to find and click on the link "ipsum".;
             Solutions?'''
four = '''action = click_action1('a', 'Tab #3', observation)
observation, reward, terminated, truncated, info = env.step(action)'''
five = '''Objects in Image: ['a Tab #1', 'a Tab #2', 'a Tab #3', 'span c', 'span ipsum'];
             Task: 'Switch between the tabs to find and click on the link "ipsum".;
             Solutions?'''
six= '''action = click_action1('span', 'ipsum', observation)
observation, reward, terminated, truncated, info = env.step(action)'''

In [53]:
count = 0
iteration = 10
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
    llama2.add_prompt(zero, mode= 'user')
    llama2.add_prompt(one, mode= 'user')
    llama2.add_prompt(two, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(three, mode= 'user')
    llama2.add_prompt(four, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')
    llama2.add_prompt(five, mode= 'user')
    llama2.add_prompt(six, mode= 'answer')
    llama2.add_prompt("", mode= 'end')
    llama2.add_prompt("", mode= 'start')

    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=4000)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    parts = solution.split('[/INST]')
    desired_text = parts[-1].strip()
    desired_text = desired_text.strip() 
    if solution.startswith(llama2.get_prompt()[3:]):
        print("HIHIHI")
#         appended_part = solution[len(llama2.get_prompt())-1:]
#         print("appended part: ----",appended_part)
    program_list = desired_text.split('\n')
    reward = 0
    
    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            exec(cleaned_code)
            print(".........Executing.......")
            print("code_check:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=4000)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    parts = new_solution.split('[/INST]')
                    desired_text = parts[-1].strip()
                    desired_text = desired_text.strip() 
                    new_program_list = desired_text.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "vitae".


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


click_action1
.........Executing.......
code_check: action = click_action1('a', 'Tab #3', observation)
.........Executing.......
code_check: observation, reward, terminated, truncated, info = env.step(action)
hi


...

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


click_action1
.........Executing.......
code_check: action = click_action1('a', 'Tab #1', observation)
.........Executing.......
code_check: observation, reward, terminated, truncated, info = env.step(action)
hi


click_action1
.........Executing.......
code_check: action = click_action1('span', 'Neque', observation)
.........Executing.......
code_check: observation, reward, terminated, truncated, info = env.step(action)
Iteration 9


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "adipiscing.".
click_action1
.........Executing.......
code_check: action = click_action1('a', 'adipiscing', observation)
Output problem: string indices must be integers
Success Rate: 0.0


In [ ]:
4/10+2/10+5/10+0/10

In [57]:
count = 0
iteration = 1
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-medium-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
    llama2.add_prompt(task_message, mode="user")
#     llama2.add_prompt(zero, mode= 'user')
#     llama2.add_prompt(one, mode= 'user')
#     llama2.add_prompt(two, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(three, mode= 'user')
#     llama2.add_prompt(four, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(five, mode= 'user')
#     llama2.add_prompt(six, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')

    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=4000)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    parts = solution.split('[/INST]')
    desired_text = parts[-1].strip()
    desired_text = desired_text.strip() 
    if solution.startswith(llama2.get_prompt()[3:]):
        print("HIHIHI")
#         appended_part = solution[len(llama2.get_prompt())-1:]
#         print("appended part: ----",appended_part)
    program_list = desired_text.split('\n')
    reward = 0
    
    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            exec(cleaned_code)
            print(".........Executing.......")
            print("code_check:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=4000)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    parts = new_solution.split('[/INST]')
                    desired_text = parts[-1].strip()
                    desired_text = desired_text.strip() 
                    new_program_list = desired_text.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "enim,.".
HIHIHI
click_action1
Output problem: string indices must be integers
click_action1
Output problem: string indices must be integers
enter_text_action
Output problem: string indices must be integers
click_action1
Output problem: string indices must be integers
Success Rate: 0.0


In [ ]:
3/10+7/10+2/10+0+1+1+0+0

In [58]:
count = 0
iteration = 1
incorrect_program_list = []

for i in range(iteration):
    print('==============================')
    print(f"Iteration {i}")
    env = gym.make("miniwob/click-tab-2-easy-v1", render_mode="human")
    observation, info = env.reset()
    task = observation["utterance"]
    print('Task:', task)

    llama2.restore_prompt()
#     llama2.add_prompt(task_message, mode="user")
#     llama2.add_prompt(zero, mode= 'user')
#     llama2.add_prompt(one, mode= 'user')
#     llama2.add_prompt(two, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(three, mode= 'user')
#     llama2.add_prompt(four, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')
#     llama2.add_prompt(five, mode= 'user')
#     llama2.add_prompt(six, mode= 'answer')
#     llama2.add_prompt("", mode= 'end')
#     llama2.add_prompt("", mode= 'start')

    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')

    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length=4000)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
    
    parts = solution.split('[/INST]')
    desired_text = parts[-1].strip()
    desired_text = desired_text.strip() 
    if solution.startswith(llama2.get_prompt()[3:]):
        print("HIHIHI")
#         appended_part = solution[len(llama2.get_prompt())-1:]
#         print("appended part: ----",appended_part)
    program_list = desired_text.split('\n')
    reward = 0
    
    for code in program_list:
        if code.strip() == "":
            continue
        code_check = p.check_promt(code)
        if not code_check or code_check == 'not_useful':
            continue

        try:
            cleaned_code = code_check.lstrip()
            exec(cleaned_code)
            print(".........Executing.......")
            print("code_check:",cleaned_code)
            if reward > 0:
                print('Reward:', reward)
                count += 1
                break
            elif reward < 0:
                break
            else:
                if code_check.startswith('observation'):
                    print("hi")
                    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
                    llama2.add_prompt("Objects in Image: {}; Task: {} Solution?".format(objects_in_the_image, task), mode='user')
                    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
                    generate_ids = model.generate(inputs.input_ids, max_length=4000)
                    new_solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True)[0]
                    if solution.startswith(llama2.get_prompt()[3:]):
                        appended_part = new_solution[len(llama2.get_prompt())-1:]
                    parts = new_solution.split('[/INST]')
                    desired_text = parts[-1].strip()
                    desired_text = desired_text.strip() 
                    new_program_list = desired_text.split('\n')
                    program_list.extend(new_program_list)
        except Exception as e:
            print('Output problem:', str(e))
            incorrect_program_list.append([task, code_check])

        if reward != 0:
            print('Reward:', reward)
            break

print(f"Success Rate: {count/iteration}")

Iteration 0


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Task: Switch between the tabs to find and click on the link "scelerisque".


KeyboardInterrupt: 

In [ ]:
17/20+5/10+0+1+0+0+not running

In [10]:
env = gym.make("miniwob/click-checkboxes-transfer-v1", render_mode="human")
observation, info = env.reset()
objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
task=observation["utterance"]

In [11]:
objects_in_the_image

['div wrap',
 'div area',
 'div boxes',
 'label ',
 'input_checkbox ch0',
 't aGnWi',
 'label ',
 'input_checkbox ch1',
 't Sq',
 'label ',
 'input_checkbox ch2',
 't l1lMtWL',
 'label ',
 'input_checkbox ch3',
 't inLT',
 'label ',
 'input_checkbox ch4',
 't JpDFjO',
 'button Submit']

In [12]:
task

'Select inLT and click Submit.'

In [ ]:
'''Objects in Image: 'div wrap', 'div area','div boxes','label ','input_checkbox ch0','t aGnWi', 'label ','input_checkbox ch1','t Sq','label ','input_checkbox ch2','t l1lMtWL','label ','input_checkbox ch3','t inLT','label ','input_checkbox ch4','t JpDFjO','button Submit' ;
             Task: Select inLT and click Submit.;
             Solutions? 
action1 = click_action1('input_checkbox', 'ch3', observation)
observation, reward, terminated, truncated, info = env.step(action1)

action11 = click_action1('button', 'Submit', observation)
observation, reward, terminated, truncated, info = env.step(action1)'''

In [29]:
env = gym.make("miniwob/click-checkboxes-v1", render_mode="human")
observation, info = env.reset()
objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
task=observation["utterance"]

In [30]:
objects_in_the_image

['div wrap',
 'div area',
 'div boxes',
 'label ',
 'input_checkbox ch0',
 't eBtuH8B',
 'label ',
 'input_checkbox ch1',
 't OqZ',
 'button Submit']

In [31]:
task

'Select eBtuH8B and click Submit.'

In [ ]:
'''Objects in Image: 'div wrap','div area','div boxes','label ','input_checkbox ch0','t eBtuH8B','label ','input_checkbox ch1','t OqZ','button Submit' ;
             Task: Select eBtuH8B and click Submit.;
             Solutions? 
action1 = click_action1('input_checkbox', 'ch0', observation)
observation, reward, terminated, truncated, info = env.step(action1)

action11 = click_action1('button', 'Submit', observation)
observation, reward, terminated, truncated, info = env.step(action1)'''

In [14]:
env = gym.make("miniwob/click-dialog-2", render_mode="human")
observation, info = env.reset()
objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
task=observation["utterance"]

In [15]:
objects_in_the_image

['div wrap',
 'div area',
 'div ',
 'div ',
 'span ui-id-1',
 'button ',
 'span ',
 't Close',
 'div dialog',
 'p Tincidunt massa. Aliquam eu, erat pellentesque.',
 'div ',
 'div ',
 'button Cancel',
 'button OK',
 'div ',
 'div ',
 'div ',
 'div ',
 'div ',
 'div ',
 'div ',
 'div ']

In [16]:
task

'Click the button in the dialog box labeled "OK".'

In [ ]:
'''Objects in Image: 'div wrap', 'div area','div ','div ','span ui-id-1','button ','span ','t Close','div dialog','p Tincidunt massa. Aliquam eu, erat pellentesque.','div ','div ','button Cancel','button OK','div ','div ','div ','div ','div ','div ','div ','div ' ;
             Task: Click the button in the dialog box labeled "OK".;
             Solutions? 
action1 = click_action1('button', 'OK', observation)
observation, reward, terminated, truncated, info = env.step(action1)
'''

In [18]:
env = gym.make("miniwob/click-dialog", render_mode="human")
observation, info = env.reset()
objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
task=observation["utterance"]

In [19]:
objects_in_the_image

['div wrap',
 'div area',
 'div ',
 'div ',
 'span ui-id-1',
 'button ',
 'span ',
 't Close',
 'div dialog',
 'p Massa nibh et ultrices.',
 'div ',
 'div ',
 'div ',
 'div ',
 'div ',
 'div ',
 'div ',
 'div ']

In [20]:
task

'Close the dialog box by clicking the "x".'

In [ ]:
'''Objects in Image: 'div wrap', 'div area','div ','div ','span ui-id-1', 'button ', 'span ','t Close','div dialog','p Massa nibh et ultrices.','div ','div ','div ','div ','div ','div ','div ','div ' ;
             Task: Close the dialog box by clicking the "x".;
             Solutions? 
action1 = click_action1('t', 'Close', observation)
observation, reward, terminated, truncated, info = env.step(action1)
'''

In [22]:
env = gym.make("miniwob/click-link", render_mode="human")
observation, info = env.reset()
objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
task=observation["utterance"]

In [23]:
objects_in_the_image

['div wrap',
 'div area',
 't Sodales tellus curabitur a',
 'span amet.',
 'span Scelerisque.',
 't Et',
 'span libero',
 'span porttitor',
 't eget',
 't pulvinar rhoncus odio velit. Amet',
 't semper faucibus. Arcu.',
 'span Ullamcorper',
 't justo.']

In [24]:
task

'Click on the link "porttitor".'

In [ ]:
'''Objects in Image: 'div wrap', 'div area','t Sodales tellus curabitur a','span amet.','span Scelerisque.','t Et','span libero','span porttitor','t eget','t pulvinar rhoncus odio velit. Amet','t semper faucibus. Arcu.','span Ullamcorper','t justo.' ;
             Task: Click on the link "porttitor".;
             Solutions? 
action1 = click_action1('span', 'porttitor', observation)
observation, reward, terminated, truncated, info = env.step(action1)
'''

In [26]:
env = gym.make("miniwob/click-option", render_mode="human")
observation, info = env.reset()
objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
task=observation["utterance"]

In [27]:
objects_in_the_image

['div wrap',
 'div area',
 'div boxes',
 'label ',
 'input_radio ch0',
 't lq0ED',
 'label ',
 'input_radio ch1',
 't CiAYduf',
 'label ',
 'input_radio ch2',
 't 6g',
 'button Submit']

In [28]:
task

'Select lq0ED and click Submit.'

In [ ]:
'''Objects in Image: 'div wrap','div area','div boxes','label ','input_radio ch0','t lq0ED','label ','input_radio ch1','t CiAYduf','label ','input_radio ch2','t 6g','button Submit' ;
             Task: Select lq0ED and click Submit.;
             Solutions? 
action1 = click_action1('input_radio', 'ch0', observation)
observation, reward, terminated, truncated, info = env.step(action1)
'''

In [32]:
env = gym.make("miniwob/click-collapsible", render_mode="human")
observation, info = env.reset()
objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
task=observation["utterance"]

In [33]:
objects_in_the_image

['div wrap', 'div area', 'h3 Section #34', 'p ui-id-3', 'button Submit']

In [34]:
task

'Expand the section below and click submit.'

In [ ]:
'''Objects in Image: 'div wrap', 'div area', 'h3 Section #34', 'p ui-id-3', 'button Submit' ;
             Task: Expand the section below and click submit.;
             Solutions? 
action1 = click_action1('h3', 'Section #34', observation)
observation, reward, terminated, truncated, info = env.step(action1)
'''

In [57]:
count = 0
iteration = 1
for i in tqdm(range(iteration)):
    p = filter_prompt()
    llama2.restore_prompt()
#     llama2.add_prompt(task_message_1,mode="user")
#     llama2.add_prompt(example,mode="user")
    env = gym.make("miniwob/click-link-v1", render_mode="human")
    observation, info = env.reset()
    objects_in_the_image = get_DOM_element(observation, useful_tag, useful_classes)
    task=observation["utterance"]
    llama2.add_prompt("Objects in Image: {}; \
                       Task:{}'  \
                       Solution?".format(objects_in_the_image,task))
    
    inputs = tokenizer(llama2.get_prompt(), return_tensors="pt").to('cuda:0')
    generate_ids = model.generate(inputs.input_ids, max_length = 4000,do_sample=False)
    solution = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    print(solution)
    if solution.startswith(llama2.get_prompt()[3:]):
        # Extract the appended part
        appended_part = solution[len(llama2.get_prompt())-1:]
        
    program_list = appended_part.split('\n') 
    p = filter_prompt()

    for code in program_list:
        # print(code)
        if code==" "or code == "\n":
            continue
        # print(code)
        code_check = p.check_promt(code)
        # print(code_check)
        if code_check == False or code_check == 'not_useful':
            # print('invalid program: ' + code)
            continue
        cleaned_code = code_check.lstrip()
        print('Here is cleaned_code')
        print(cleaned_code)
        try:
            exec(cleaned_code)
            
        except:
            print('output problem')
            continue
        
        try:
            if reward > 0:
                count += 1
                reward = 0
                break
        except:
            continue
        
print(count/iteration)

  0%|                                                     | 0/1 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
100%|████████████████████████████████████████████| 1/1 [04:43<00:00, 283.81s/it]

[INST] <<SYS>> { You are designed to generate programs to solve a wide range of complex web interface tasks.
You should be able to generate the program using either one or a composition of predefined action functions 
along with general python codes to solve different tasks. You should not conversate with human in any context.
 
You should only use the functions provided herewith in the function description. 
Here is the list for the pre-defined functions [click_action1, enter_text_action, coord_click_action, count_type]

To use a function, please refer to the Name, Input, Output, Description of the functions, and usage examples below. 
Action functions should be called correctly in the solution. 

Function Name: click_action1
Input: tag or element, id or text, observation
Output: create_click_element_action
Description: useful when you want to click on an element in the web interface. This functions cannot be 
            generalized on names. Normally first input is one of tag or ele